# ---------------------------------------


# Usando Dataset Projeto

## Bibliotecas

In [3]:
!pip install rank_bm25
!pip install unidecode
!pip install --upgrade gensim
!pip install pandas
!pip install nltk

Requirement already up-to-date: gensim in /home/p1ntsu/anaconda3/envs/projetoFAQ/lib/python3.9/site-packages (3.8.3)
     |████████████████████████████████| 9.3 MB 6.5 MB/s eta 0:00:01
  Using cached pytz-2020.4-py2.py3-none-any.whl (509 kB)
  Using cached nltk-3.5.zip (1.4 MB)
  Using cached click-7.1.2-py2.py3-none-any.whl (82 kB)
     |████████████████████████████████| 302 kB 729 kB/s eta 0:00:01
     |████████████████████████████████| 732 kB 4.9 MB/s eta 0:00:01
     |████████████████████████████████| 69 kB 655 kB/s eta 0:00:01
  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434674 sha256=38c061df2808722bb0c3e0e7b1dcb69d28486b82db2d04ab325ce33c0fef3212
  Stored in directory: /home/p1ntsu/.cache/pip/wheels/13/ae/bb/5e2a232ebaa1d2f38dd5f587e9fc4cf6ccb12758d14dac14d8
Successfully built nltk


In [4]:
import pandas as pd
import numpy as np
import json
import nltk
import string
from rank_bm25 import BM25Okapi
from unidecode import unidecode
import gensim
from gensim.summarization.bm25 import BM25

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/p1ntsu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Abertura do dataset

In [6]:
with open('../DataSets/Projeto_Repositorio/repositorio.csv','r') as f:
    df = pd.read_csv(f)
df.head()

,Classificação (FioCruz),Sentença,Resposta
0,O que é ?,O que é o novo coronavírus?,Os coronavírus causam infecções respiratórias ...
1,O que é ?,Por que a doença causada pelo novo vírus receb...,"Desde o início de fevereiro, a Organização Mun..."
2,Boatos,A Organização Mundial de Saúde (OMS) disse que...,Não. A Organização Mundial da Saúde não disse ...
3,Boatos,Diante da falta de álcool gel podemos colocar ...,Preparações caseiras não possuem a certificaçã...
4,Boatos,"Chá de abacate com hortelã, mel, uísque ou out...","Até o momento, não há nenhum medicamento, subs..."


In [5]:
new_df = df[df['Sentença'] != 'Sentença'].copy()
new_df.shape

(722, 3)

## Pre-processamento do dataset

In [ ]:
new_df.columns = ['classe','pergunta','resposta']
new_df.drop(['classe'],axis=1,inplace=True)
new_df.head()

,pergunta,resposta
0,O que é o novo coronavírus?,Os coronavírus causam infecções respiratórias ...
1,Por que a doença causada pelo novo vírus receb...,"Desde o início de fevereiro, a Organização Mun..."
2,A Organização Mundial de Saúde (OMS) disse que...,Não. A Organização Mundial da Saúde não disse ...
3,Diante da falta de álcool gel podemos colocar ...,Preparações caseiras não possuem a certificaçã...
4,"Chá de abacate com hortelã, mel, uísque ou out...","Até o momento, não há nenhum medicamento, subs..."


In [ ]:
new_df.isna().sum()

pergunta    24
resposta    24
dtype: int64

In [ ]:
new_df.dropna(inplace = True)
new_df.reset_index(inplace= True,drop=True)
new_df.head()

,pergunta,resposta
0,O que é o novo coronavírus?,Os coronavírus causam infecções respiratórias ...
1,Por que a doença causada pelo novo vírus receb...,"Desde o início de fevereiro, a Organização Mun..."
2,A Organização Mundial de Saúde (OMS) disse que...,Não. A Organização Mundial da Saúde não disse ...
3,Diante da falta de álcool gel podemos colocar ...,Preparações caseiras não possuem a certificaçã...
4,"Chá de abacate com hortelã, mel, uísque ou out...","Até o momento, não há nenhum medicamento, subs..."


In [ ]:
perguntas = new_df['pergunta']
for i in range(perguntas.size):
    perguntas[i] = unidecode(perguntas[i]).lower()
new_df['pergunta'] = perguntas
new_df.head()

,pergunta,resposta
0,o que e o novo coronavirus?,Os coronavírus causam infecções respiratórias ...
1,por que a doenca causada pelo novo virus receb...,"Desde o início de fevereiro, a Organização Mun..."
2,a organizacao mundial de saude (oms) disse que...,Não. A Organização Mundial da Saúde não disse ...
3,diante da falta de alcool gel podemos colocar ...,Preparações caseiras não possuem a certificaçã...
4,"cha de abacate com hortela, mel, uisque ou out...","Até o momento, não há nenhum medicamento, subs..."


In [ ]:
print(("Dados duplicados: %d")%(new_df.duplicated(['pergunta']).sum()))
new_df.drop_duplicates(['pergunta'],inplace=True)
new_df.reset_index(inplace= True,drop = True)
print("Shape dataset: {}".format(new_df.shape))

Dados duplicados: 133
Shape dataset: (565, 2)


### Salvando dataset


In [ ]:
new_df.to_csv('../DataSets/Projeto_Repositorio/repositorio_projeto.csv')

### Abrindo dataset

In [8]:
new_df = pd.read_csv('../DataSets/Projeto_Repositorio/repositorio_projeto.csv')

In [9]:
perguntas = new_df['pergunta']

In [10]:
document = []
for pergunta,i in zip(perguntas,range(perguntas.size)):
    try:
        tokens = nltk.word_tokenize(pergunta,language = 'portuguese')
        tokens = [w for w in tokens if not w in string.punctuation]
        document.append(tokens)
    except:
        print(pergunta, i)
document = np.array(document)

<ipython-input-10-c46f98ed8fea>:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  document = np.array(document)


## Salvando amostra

In [ ]:
sample = np.random.choice(document,size=50,replace=False)

In [ ]:
with open('../DataSets/Projeto_Repositorio/Amostra/amostra.json','w') as outfile:
    json.dump(sample,outfile)

## Abrindo amostra


In [7]:
with open('../DataSets/Projeto_Repositorio/Amostra/amostra.json','r') as f:
    sample = json.load(f)

## Utilizando BM25

In [11]:
bm25 = BM25(document)
#bm25 = BM25Okapi(document)

In [14]:
posicao =  1
try:
    query = sample[posicao]
    print('Query:'," ".join(query) + '?','\n')
    score = bm25.get_scores(query)
    top_n = np.argsort(score)[::-1][2:6] # Apresentar as 3 melhores assimilações
    for i in top_n:
        print("Pergunta: {} | Score BM25: {:.2f}".format(perguntas[i],score[i])) # Print da pergunta similiar e o score do BM25
except:
    print("Posicao invalida")

Query: o que sao coronavirus? 

Pergunta: o que sao testes rapidos (igm/igg)? | Score BM25: 4.49
Pergunta: o que sao os testes rapidos? | Score BM25: 4.49
Pergunta: o que sao testes rt- pcr? | Score BM25: 4.49
Pergunta: quais sao as complicacoes do novo coronavirus? | Score BM25: 4.42


### Analise de metricas

In [15]:
df_avaliado = pd.read_csv('../DataSets/Projeto_Repositorio/repositorio_avaliado_v2.csv')
df_avaliado.head()

,pergunta,assimilacao_um,nota_um,assimilacao_dois,nota_dois,assimilacao_tres,nota_tres
0,quais sao os sintomas do novo coronavirus?,quais os sintomas do coronavirus?,100,quais sao os sintomas?,100,quais sao as complicacoes do novo coronavirus?,75
1,o que sao coronavirus?,o que e o novo coronavirus?,100,o que sao os coronavirus?,100,o que e covid-19?,100
2,qual o plano da secretaria da saude para monit...,como a rede municipal de saude se preparou par...,60,qual a orientacao para os profissionais da sau...,0,o ministerio da saude tem uma plataforma para ...,80
3,quem precisa ficar em isolamento?,casos suspeitos tem sido mantidos em isolament...,60,o que significa isolamento domiciliar?,10,que cuidados devo ter se estiver em isolamento?,0
4,como e feito o diagnostico do covid-19?,como e feito o diagnostico da covid-19?,100,como e feito o diagnostico?,100,como e feito o tratamento contra o coronavirus?,0


In [16]:
media_um = df_avaliado['nota_um'].mean()
desvio_um = df_avaliado['nota_um'].std()
media_dois = df_avaliado['nota_dois'].mean()
desvio_dois = df_avaliado['nota_dois'].std()
media_tres = df_avaliado['nota_tres'].mean()
desvio_tres = df_avaliado['nota_tres'].std()
print("Media 1: {} +- {:.2f}\nMedia 2: {} +- {:.2f}\nMedia 3: {} +- {:.2f}".format(media_um,desvio_um,media_dois,desvio_dois,media_tres,desvio_tres))
print("Media geral: {:.2f}".format(np.array([media_um,media_dois,media_tres]).mean()))

Media 1: 56.7 +- 40.46
Media 2: 40.26 +- 42.96
Media 3: 25.1 +- 37.78
Media geral: 40.69


In [17]:
df_avaliado.drop(['pergunta','assimilacao_um','assimilacao_dois','assimilacao_tres'],axis = 1,inplace = True)

In [18]:
descriminante = 0 # Qualquer valor maior que... 
matrix_avaliacao = np.zeros(df_avaliado.shape,dtype = int)
matrix_cumali = np.zeros(df_avaliado.shape,dtype=int) # Cumulative Gain@k (CG@k)
matrix_disc_cumali = np.zeros(df_avaliado.shape) # Discounted Cumulative Gain@k (DCG@k)
matrix_disc_i_cumali = np.zeros(df_avaliado.shape) # Ideal Discounted Cumulative Gain@k (IDCG@k)
df_array = df_avaliado.values
for posi in range(df_array.shape[0]):
    soma_cg = 0
    soma_dcg = 0
    soma_idcg = 0
    sorted_matrix = np.sort(df_array[posi])[::-1]
    for posi_interno in range(df_array[posi].size):

        #----------------------------------------------#
        if(df_array[posi][posi_interno] > descriminante):
            matrix_avaliacao[posi][posi_interno] = 1
        #----------------------------------------------#

        #----------------------------------------------#
        soma_cg += df_array[posi][posi_interno]
        matrix_cumali[posi][posi_interno] = soma_cg # (CG@k)

        #----------------------------------------------#
        soma_dcg += (df_array[posi][posi_interno]/np.log2(posi_interno + 2))
        matrix_disc_cumali[posi][posi_interno] = soma_dcg # (DCG@k)

        #----------------------------------------------#
        soma_idcg += (sorted_matrix[posi_interno]/np.log2(posi_interno + 2))
        matrix_disc_i_cumali[posi][posi_interno] = soma_idcg # (IDCG@k)
      
    

In [19]:
matrix_pre = np.zeros(df_avaliado.shape) # precision@k
matrix_recall = np.zeros(df_avaliado.shape) # recall@k
for posi in range(matrix_avaliacao.shape[0]):
    soma = matrix_avaliacao[posi].sum()
    if(soma != 0):
        tp = 0
        fp = 0
        for i,j in zip(matrix_avaliacao[posi],range(matrix_avaliacao[posi].size)):
            if(i == 1):
                tp += 1
                matrix_pre[posi][j] = round((tp/(tp+fp)),2)
                matrix_recall[posi][j] = round((tp/soma),2)
            else:
                fp += 1
                matrix_pre[posi][j] = round((tp/(tp+fp)),2)
                matrix_recall[posi][j] = round((tp/soma),2)

In [20]:
matrix_ap = np.zeros(df_avaliado.shape[0]) # average precision
for posi in range(matrix_avaliacao.shape[0]):
    soma = 0
    for i in range(matrix_avaliacao[posi].size):
        if(matrix_avaliacao[posi][i] == 1):
            soma += matrix_pre[posi][i]
    matrix_ap[posi] = round((soma/matrix_avaliacao[posi].size),2)


In [21]:
matrix_ndgc = np.zeros(df_avaliado.shape) # Normalized Discounted Cumulative Gain
for posi in range(matrix_ndgc.shape[0]):
    for posi_interno in range(matrix_ndgc.shape[1]):
        if(matrix_disc_i_cumali[posi][posi_interno] != 0):
            matrix_ndgc[posi][posi_interno] = round((matrix_disc_cumali[posi][posi_interno]/matrix_disc_i_cumali[posi][posi_interno]),2)

In [22]:
MAP = matrix_ap.mean() # mean average precision
MAP_std = matrix_ap.std() # mean average precision std
print("MAP: {:.2f} +- {:.2f}".format(MAP,MAP_std))

MAP: 0.56 +- 0.34


In [23]:
df_ndgc = pd.DataFrame(matrix_ndgc,columns=['N@1','N@2','N@3'])
df_ndgc.head()

,N@1,N@2,N@3
0,1.00,1.00,1.00
1,1.00,1.00,1.00
2,0.75,0.51,0.85
3,1.00,1.00,1.00
4,1.00,1.00,1.00


In [24]:
dici = {'MAP': [round(MAP,4)], 'N@1' : [round(df_ndgc['N@1'].mean(),4)], 'N@2' : [round(df_ndgc['N@2'].mean(),4)], 'N@3' : [round(df_ndgc['N@3'].mean(),4)]}
new_df_ndgc = pd.DataFrame(dici,index=['BM25'])
new_df_ndgc

,MAP,N@1,N@2,N@3
BM25,0.5596,0.7488,0.7742,0.8332


In [29]:
new_df_ndgc.to_csv('../Tabelas/metricas_bm25.csv')

In [30]:
new_df_ndgc.to_latex('../Tabelas/metricas_bm25.tex')

### Correcao manual valores nulos 

In [25]:
df_avaliado = df_avaliado[df_avaliado['nota_um'] == 0]
df_avaliado = df_avaliado[df_avaliado['nota_dois'] == 0]
df_avaliado = df_avaliado[df_avaliado['nota_tres'] == 0]
df_avaliado.head()

,nota_um,nota_dois,nota_tres
11,0,0,0
20,0,0,0
29,0,0,0
35,0,0,0
36,0,0,0


In [26]:
print("Tamanho do DF avaliado: {}".format(df_avaliado.shape[0]))

Tamanho do DF avaliado: 5


In [28]:
df_avaliado.values[4]

array([0, 0, 0])

In [ ]:
#@title Texto de título padrão { run: "auto" }
variable_name = "profissio" #@param {type:"string"}
for pergunta in perguntas:
    if(variable_name in pergunta):
        print(pergunta)

qual a orientacao para os profissionais da saude quando retornam a seus domicilios?
as profissionais de saude gravidas correm maior risco de resultados adversos se cuidarem de pacientes com covid-19?
como e a prevencao contra o coronavirus para os profissionais de saude?
por que e importante o municipio fazer a notificacao dentro de 24 horas? quais providencias o profissional deve adotar?
os testes rapidos sao de uso profissional?
o profissional de enfermagem pode utilizar mascara caseira em servicos de saude?
se um profissional da equipe testar positivo para covid-19, o que a equipe devera fazer?
quais os cuidados a serem adotados pelos profissionais de enfer-magem que realizam assistencia em atencao domiciliar? e se o paciente estiver com quadro suspeito ou confirmado?
posso atuar sem carteira de identidade profissional (cip)?
a minha carteira de identidade profissional esta vencida. o que devo fazer?
como sei que o profissional de saude esta tomando a precaucao correta contra o covi

In [ ]:
#query = np.array(question_test)[6]
query = question_test.iloc[5]
print(query,'\n')
query = nltk.word_tokenize(unidecode(query).lower(),language = 'portuguese')
query = [w for w in query if not w in string.punctuation]
score = bm25.get_scores(query)

top_n = np.argsort(score)[::-1][:5] # Apresentar as 5 melhores assimilações
for i in top_n:
    print("Pergunta: {} | Score BM25: {:.2f}".format(perguntas[i],score[i])) # Print da pergunta similiar e o score do BM25

quantas mascaras de tecido devo ter? 

Pergunta: como o usuario deve proceder para ter esse tipo de atendimento? | Score BM25: 7.47
Pergunta: quais as recomendacoes para quem precisar viajar neste momento? | Score BM25: 6.97
Pergunta: quantoslaboratorios publicos fazem o exame de deteccao em pernambuco? | Score BM25: 6.63
Pergunta: o covid-19 e  transmitido pelo ar? | Score BM25: 5.78
Pergunta: o que sao medidas de higiene e etiqueta respiratoria?  | Score BM25: 5.08
